# Text classifiction with LLMs

Goal with this notebook is to use LLM for text classficiation. 

##### Current status: 3. Working on fine-tuning a LLM.

Steps in this process:
1. Dataset and LLM selection
2. Preprocessing with tokenizing text, encoding labels, splittling into tran/test sets
3. *Fine-tuning LLM for the specific task using different techniques like gradient descent
4. Evaluate the results - have to figure this out
5. Experiment with different models, hyerparameters, etc.
6. Create a Keynote and a logo on Canva 

Code from: [HF examples](https://github.com/huggingface/notebooks/blob/main/transformers_doc/en/training.ipynb)

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [3]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\nansc\miniconda3\envs\pytorch_may2024\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 50000/50000 [00:13<00:00, 3742.39 examples/s]


In [9]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=2)



c:\Users\nansc\miniconda3\envs\pytorch_may2024\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [22]:
from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

c:\Users\nansc\miniconda3\envs\pytorch_may2024\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
# !pip install evaluate
# !pip install scikit-learn

In [24]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

  2%|▏         | 3/189 [00:36<37:09, 11.99s/it]